In [1]:
%pip install torch torchvision pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import os

# =============== Step 1: Load Pre-trained InceptionV3 Model ===============
model = models.inception_v3(pretrained=True)
model.eval()

# =============== Step 2: Define Preprocessing ===============
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet standards
        std=[0.229, 0.224, 0.225]
    )
])

# =============== Step 3: Load Image ===============

# Option 1: Load from URL
def load_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert("RGB")
        return img
    except Exception as e:
        print("Error loading image from URL:", e)
        return None

# Option 2: Load local image
def load_image_from_file(file_path):
    try:
        img = Image.open(file_path).convert("RGB")
        return img
    except Exception as e:
        print("Error loading local image:", e)
        return None

# Choose one of the options below:
# 1. From URL
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Juvenile_Ragdoll.jpg/640px-Juvenile_Ragdoll.jpg"
img = load_image_from_url(img_url)

# 2. From local file (uncomment to use local file)
# img_path = "your_image.jpg"
# img = load_image_from_file(img_path)

if img is None:
    print("Image loading failed. Exiting.")
    exit()

# =============== Step 4: Preprocess and Predict ===============
img_t = preprocess(img)
batch_t = torch.unsqueeze(img_t, 0)  # add batch dimension

with torch.no_grad():
    output = model(batch_t)

# =============== Step 5: Decode Predictions ===============
# Download labels from ImageNet if not available
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels_path = "imagenet_classes.txt"

if not os.path.exists(labels_path):
    with open(labels_path, "w") as f:
        f.write(requests.get(LABELS_URL).text)

# Read class names
with open(labels_path) as f:
    categories = [line.strip() for line in f.readlines()]

# Softmax and Top-5 predictions
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)

print("\nTop-5 Predictions:")
for i in range(top5_prob.size(0)):
    print(f"{categories[top5_catid[i]]}: {top5_prob[i].item():.4f}")


c:\Users\Programming\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Programming\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Error loading image from URL: 403 Client Error: Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy for url: https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Juvenile_Ragdoll.jpg/640px-Juvenile_Ragdoll.jpg
Image loading failed. Exiting.


TypeError: Unexpected type <class 'NoneType'>

: 

In [3]:
import requests
from PIL import Image
from io import BytesIO

# Use a working image URL
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Juvenile_Ragdoll.jpg/640px-Juvenile_Ragdoll.jpg"

# Download the image
response = requests.get(url)

# Check if image downloaded successfully
if response.status_code == 200:
    try:
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img.show()  # optional: show image to verify
    except Exception as e:
        print("Failed to open image:", e)
else:
    print("Failed to download image. Status code:", response.status_code)


Failed to download image. Status code: 403
